# Introduction to `CVXPY`

## Detour: Symbolic Convexity Checking Using `SymPy`
$\newcommand{\ds}{\displaystyle}$
For differentiable function $f$, check if its Hessian is positive-semidefinite: $\;\ds\nabla^2 f\succcurlyeq 0$
- $\ds f(x, y) = x \log\frac{x}{y},\quad x > 0,\quad y > 0$

In [ ]:
from sympy.abc import w, x, y, z, a  
from sympy import ordered, hessian, log, exp, Symbol
from sympy import init_printing
init_printing(use_latex='mathjax')

expr = x * log(x / y)
v = list(ordered(expr.free_symbols))
display(hessian(expr, v))
hessian(expr, v).eigenvals()

### Your Turn: Check the Convexity of the Following Functions
- $\ds f(x) = x\log x$, $\quad x > 0$

In [ ]:
expr = x * log(x)
v = list(ordered(expr.free_symbols))
display(hessian(expr, v))
hessian(expr, v).eigenvals()

- $\ds f(x, y) = \frac{x^2}{y}$, $\quad y > 0$

In [ ]:
expr = x**2 / y
v = list(ordered(expr.free_symbols))
display(hessian(expr, v))
hessian(expr, v).eigenvals()

- $\ds f(x, y, z) = \log\,(e^{x} + e^{y} + e^{z})$

In [ ]:
expr = log(exp(x) + exp(y) + exp(z))
v = list(ordered(expr.free_symbols))
H = hessian(expr, v)
display(H)
display(H.eigenvals())
display(H.det())
#H.minor_submatrix(1, 2).det()

- $f(x, y) = xy\;$ on $\;\mathbb{R}^2_{++}$

In [ ]:
expr = x * y
v = list(ordered(expr.free_symbols))
display(hessian(expr, v))
hessian(expr, v).eigenvals()

- $\ds f(x, y) = \frac{1}{xy}\;$ on $\;\mathbb{R}^2_{++}$

In [ ]:
expr = 1 / (x * y)
v = list(ordered(expr.free_symbols))
display(hessian(expr, v))
hessian(expr, v).eigenvals()

- $\ds f(x, y) = \frac{x}{y}\;$ on $\;\mathbb{R}^2_{++}$

In [ ]:
expr = x / y
v = list(ordered(expr.free_symbols))
display(hessian(expr, v))
hessian(expr, v).eigenvals()

- $\ds f(x, y) = x^\alpha y^{1 - \alpha}\;$ on $\;\ds\mathbb{R}^2_{++}$ where $\ds 0\leqslant\alpha\leqslant 1$.

In [ ]:
from sympy.physics.units.quantities import Quantity
a = Quantity('a')
expr = x**a * y**(1 - a)
v = list(ordered(expr.free_symbols))
display(hessian(expr, v))
hessian(expr, v).eigenvals()

## First Encounter

`CVXPY` [official site](https://www.cvxpy.org/index.html) 

In [ ]:
import cvxpy as cp

# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = cp.Minimize((x - y)**2)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  
# Return optimal values
print("status:", prob.status, "\noptimal value:", prob.value, "\noptimal variables:", x.value, y.value)

### In Practice

In [ ]:
import cvxpy as cp

# Create the optimization variable x of length 2.
x = cp.Variable(2)
prob = cp.Problem(cp.Minimize((x[0] - x[1])**2), [x[0] + x[1] == 1, x[0] - x[1] >= 1])
prob.solve()  
# Return optimal values
print("status:", prob.status, "\noptimal value:", prob.value, "\noptimal variables:", x.value)

## Showcases

### Linear Programming (LP)

Standard form
$$\begin{array}{ll}
    \mbox{minimize}   & c^\top x \\
    \mbox{subject to} & Ax \preccurlyeq b.
    \end{array}$$
where $A \in \mathbb{R}^{m \times n}$, $b \in \mathbb{R}^m$, and $c \in \mathbb{R}^n$; $x \in \mathbb{R}^{n}$ is the optimization variable.

In [ ]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random non-trivial linear program.
m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A@x0 + s0
c = -A.T@lamb0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x), [A@x <= b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value, "\nA solution x is", x.value)

### Your Turn: LP Formulations 
\begin{align*}
  \text{maximize}\quad& 5 x_1 + 4 x_2 + 3 x_3 \\
  \text{subject to}\quad& 2 x_1 + 3 x_2 + x_3 \leqslant 5 \\
  \qquad\qquad\quad& 4 x_1 + x_2 + 2 x_3 \leqslant 11 \\
  \qquad\qquad\quad& 3 x_1 + 4 x_2 + 2 x_3 \leqslant 8 \\
  \qquad\qquad\quad& x_1\geqslant 0,\quad x_2\geqslant 0,\quad x_3\geqslant 0
\end{align*}
- The maximum: $13$. The optimizer: $(2,\,0,\,1)$. Using two ways (Index & Matrix) to do this.
- You may need `x = cp.Variable(3, nonneg=True)` and `np.round()`.


In [ ]:
import numpy as np
import cvxpy as cp

x = cp.Variable(3, nonneg=True)

A = np.array([[2, 3, 1], [4, 1, 2], [3, 4, 2]])
c = np.array([5, 4, 3])
b = np.array([5, 11, 8])

# TODO: Index & Matrix
prob = cp.Problem(cp.Maximize(c@x), [A@x <= b])
#

prob.solve()
print(np.round(prob.value))
print(np.round(x.value))

### 0-1 Knapsack Problem

- A set of $n$ items number from $1$ up to $n$, each with a weight $w_i$ and a value $v_i$. The maximum weight capacity of the knapsack is $W$ and the number $x_i$ of each kind of item inside is zero or one.
- The mathematical formulation of this 0-1 knapsack problem is:
  \begin{align*}
    \text{maximize}\quad&\sum_i v_i x_i\\
    \text{subject to}\quad&\sum_i w_i x_i\leqslant W\;\text{ and }\;x_i\in\{0, 1\},\;i=1,\,2,\,\ldots,\,n
  \end{align*}
- An example (Martello-Toth Example 2.1):
  \begin{align*}
   w &= [2,\,20,\,20,\,30,\,40,\,30,\,60,\,10] \\
   v &= [15,\,100,\,90,\,60,\,40,\,15,\,10,\,1] \\
   W &= 102
  \end{align*}
  The maximum: $280$, the optimal solution $\ds x = [1,\,1,\,1,\,1,\,0,\,1,\,0,\,0]$.

In [ ]:
import cvxpy as cp
v = [15, 100, 90, 60, 40, 15, 10, 1]
w = [2, 20, 20, 30, 40, 30, 60, 10]
W = 102
x = cp.Variable(len(w), boolean=True)
prob = cp.Problem(cp.Maximize(cp.sum(v @ x)), [cp.sum(w @ x) <= W])
prob.solve()
print(prob.value)
print(x.value)

Here is [another example](https://towardsdatascience.com/can-chatgpt-solve-knapsack-problems-1a9a388c4caf) using dynamic programming; below is the integer programming solution.

In [ ]:
import cvxpy as cp
v = [
    156, 59, 61, 129, 64, 158, 87, 77, 157, 144, 156, 127, 74, 157, 91, 124,
    128, 95, 118, 127, 111, 67, 108, 139, 79, 95, 86, 155, 120, 112, 76, 116,
    79, 100, 80, 120, 117, 93, 111, 67, 55, 113, 105, 119, 171, 128, 72, 133,
    60, 86, 97, 151, 77, 75, 97, 80, 133, 67, 65, 86, 85, 157, 128, 102, 63,
    125, 69, 53, 135, 109, 127, 109, 125, 80, 68, 120, 75, 79, 86, 100, 89, 58,
    80, 33, 118, 51, 66, 101, 77, 146, 61, 108, 98, 64, 115, 94, 123, 124, 164,
    140
]
w = [
    15, 11, 12, 16, 15, 7, 12, 9, 9, 11, 10, 14, 12, 10, 11, 11, 14, 9, 10, 7,
    2, 11, 12, 7, 16, 5, 10, 9, 14, 14, 10, 11, 7, 4, 8, 10, 13, 13, 8, 9, 6, 5,
    4, 15, 8, 8, 6, 12, 5, 9, 7, 11, 8, 6, 9, 11, 10, 10, 8, 8, 7, 8, 7, 4, 10,
    8, 5, 11, 7, 10, 12, 10, 13, 6, 11, 7, 7, 8, 9, 10, 6, 12, 11, 5, 14, 15, 13,
    9, 6, 13, 8, 13, 10, 12, 11, 12, 10, 15, 10, 11
]
W = 500
x = cp.Variable(len(w), boolean=True)
prob = cp.Problem(cp.Maximize(cp.sum(v @ x)), [cp.sum(w @ x) <= W])
prob.solve(solver=cp.SCIPY)
print(prob.value)
print([ii for ii, t in enumerate([int(i) for i in x.value]) if t == 1])

### Least-Squares / Linear Regression

In a least-squares / linear regression problem, we have measurements $A \in \mathbb{R}^{m \times n}$ and $b \in \mathbb{R}^m$ and seek a vector $x \in \mathbb{R}^{n}$ such that $Ax$ is close to $b$. Closeness is defined as the sum of the squared differences:
$$ \sum_{i=1}^m (a_i^\top x - b_i)^2, $$
also known as the $\ell_2$-norm squared, $\|Ax - b\|_2^2$.

We find the optimal $x$ by solving the optimization problem
$$  
    \begin{array}{ll}
    \mbox{minimize}   & \|Ax - b\|_2^2.
    \end{array}
$$
Let $x^\star$ denote the optimal $x$. The quantity $r = Ax^\star - b$ is known as the residual. If $\|r\|_2 = 0$, we have a perfect fit.

In [ ]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate data.
m = 20
n = 15
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
cost = cp.sum_squares(A@x - b)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value, "\nThe optimal x is", x.value, 
      "\nThe norm of the residual is", cp.norm(A@x - b, p=2).value)

### Your Turn: Ridge & LASSO  

- We wish to recover a sparse vector $x \in \mathbb{R}^n$ from measurements $y\in\mathbb{R}^m$. Our measurement model tells us that 
$$y = Ax + v$$
where $A \in \mathbb{R}^{m \times n}$ is known and $v \in \mathbb{R}^m$ is unknown measurement error.
- The entries of $v$ are sampled from the normal distribution with mean $0$ and standard deviation $\sigma = 1$.
- We first try to recover $x$ by solving the Ridge problem
\begin{align*}
  \text{mimimize}\quad \|Ax-y\|^2_2 + \gamma\|x\|^2_2
\end{align*}
- A more effective approach is to solve the LASSO problem
\begin{align*}
  \text{mimimize}\quad \|Ax-y\|^2_2 + \gamma\|x\|_1
\end{align*}
- Estimate $x$ from $y$ using both Ridge and LASSO regression. Your task is to provide the definitions of `ridge_loss` and `lasso_loss`.
- We first let `gamma` equals $1$. Can you find a better `gamma`?
- Suggestion: [`CVXPY` functions lookup.](https://www.cvxpy.org/tutorial/functions/index.html)

In [ ]:
# Ridge regression vs. LASSO to estimate sparse x.
import numpy as np
import cvxpy as cp
import scipy.linalg as la
import scipy.sparse as sp

np.random.seed(1)
n = 4000
m = 2000
true_x = 100 * sp.rand(n, 1, 0.1).toarray().flatten()
A = np.random.randn(m, n)
v = np.random.normal(0, 1, m)
y = A @ true_x + v

x = cp.Variable(n)
gamma = 1

ridge_loss = None # FIXME
ridge = cp.Problem(cp.Minimize(ridge_loss))
ridge.solve()
x_ridge = x.value

lasso_loss = None # FIXME
lasso = cp.Problem(cp.Minimize(lasso_loss))
lasso.solve()
x_lasso = x.value

import matplotlib.pyplot as plt
%matplotlib inline
plt.semilogy(range(n), np.sort(np.abs(true_x - x_ridge)),  label="ridge errors")
plt.semilogy(range(n), np.sort(np.abs(true_x - x_lasso)),  label="lasso errors")
plt.legend()
plt.show()

### Control Problem

- A system with a state $x_t\in\mathbb{R}^n$ that varies over the time steps $t=0,\ldots,T$, and inputs or actions $u_t\in\mathbb{R}^m$ one can use at each time step to affect the state.
- E.g. $x_t$ be the position and velocity of a rocket and $u_t$ the output of the rocket's thrusters.
- The evolution of the state is modeled as a linear dynamical system 
$$ x_{t+1} = Ax_t + Bu_t $$
where $A \in \mathbb{R}^{n\times n}$ and $B \in \mathbb{R}^{n\times m}$ are known matrices.
- Goal: Find the optimal actions $u_0,\ldots,u_{T-1}$ by solving
\begin{align*}
  \text{minimize}\quad   & \sum_{t = 0}^{T - 1}\ell(x_t, u_t) + \ell_T(x_T) \\
  \text{subject to}\quad & x_{t + 1} = A\,x_t + B\,u_t \\
  \qquad\qquad & (x_t, u_t)\in\mathcal{C},\quad x_t\in\mathcal{C}
\end{align*}
where $\ell: \mathbb{R}^n \times \mathbb{R}^m\to \mathbb{R}$ is the stage cost, $\ell_T$ is the terminal cost,
$\mathcal C$ is the state/action constraints, and $\mathcal C_T$ is the terminal constraint.
- The optimization problem is convex if the costs and constraints are convex.
- Solve a control problem with $n=8$ states, $m=2$ inputs, and horizon $T=50$.
- Matrices $A$ and $B$ and the initial state $x_0$ are randomly chosen (with $A\approx I$).
- Use the (traditional) stage cost $\ds\,\ell(x,u) = \|x\|_2^2 + \|u\|_2^2,\,$ the input constraint $\ds\|u_t\|_\infty \leqslant 1$, and the terminal constraint $x_T = 0$.
- The results below show $4$-high stack of plots showing $u_1$, $u_2$, $x_1$, and $x_2$ vs $t$. Notice that $u_t$ is saturated (i.e., $\ds\ds\|u_t\|_\infty = 1$) initially, which shows that the input constraint is meaningful.

In [ ]:
# Generate data for control problem.
import numpy as np

np.random.seed(1)
n = 8
m = 2
T = 50
alpha = 0.2
beta = 3
A = np.eye(n) - alpha * np.random.rand(n, n)
B = np.random.randn(n, m)
x_0 = beta * np.random.randn(n)

# Form and solve control problem.
import cvxpy as cp

x = cp.Variable((n, T + 1))
u = cp.Variable((m, T))

cost = 0
constr = []
for t in range(T):
    cost += cp.sum_squares(x[:, t + 1]) + cp.sum_squares(u[:, t])
    constr += [x[:, t + 1] == A @ x[:, t] + B @ u[:, t], cp.norm(u[:, t], "inf") <= 1]
# sums problem objectives and concatenates constraints.
constr += [x[:, T] == 0, x[:, 0] == x_0]
problem = cp.Problem(cp.Minimize(cost), constr)
problem.solve()

# Plot results.
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'

f = plt.figure()

# Plot (u_t)_1.
ax = f.add_subplot(411)
plt.plot(u[0, :].value)
plt.ylabel(r"$(u_t)_1$", fontsize=16)
plt.yticks(np.linspace(-1.0, 1.0, 3))
plt.xticks([])

# Plot (u_t)_2.
plt.subplot(4, 1, 2)
plt.plot(u[1, :].value)
plt.ylabel(r"$(u_t)_2$", fontsize=16)
plt.yticks(np.linspace(-1, 1, 3))
plt.xticks([])

# Plot (x_t)_1.
plt.subplot(4, 1, 3)
x1 = x[0, :].value
plt.plot(x1)
plt.ylabel(r"$(x_t)_1$", fontsize=16)
plt.yticks([-10, 0, 10])
plt.ylim([-10, 10])
plt.xticks([])

# Plot (x_t)_2.
plt.subplot(4, 1, 4)
x2 = x[1, :].value
plt.plot(range(51), x2)
plt.yticks([-25, 0, 25])
plt.ylim([-25, 25])
plt.ylabel(r"$(x_t)_2$", fontsize=16)
plt.xlabel(r"$t$", fontsize=16)
plt.tight_layout()
plt.show()

<hr>

### Quadratic Program (QP)

- An optimization problem with a quadratic objective and affine equality and inequality constraints.
- A common standard form is the following:
\begin{align*}
  \text{minimize}\quad & \frac{1}{2}\,x^\top Px + q^\top x\\
  \text{subject to}\quad & G\,x\preccurlyeq h \\
  \qquad\qquad & A\,x = b
\end{align*}
Here $P \in \mathsf{S}^{n}_+$, $q \in \mathbb{R}^n$, $G\in\mathbb{R}^{m \times n}$, $h\in\mathbb{R}^m$, $A\in\mathbb{R}^{p \times n}$, and $b\in\mathbb{R}^p$ are problem data and $x\in\mathbb{R}^{n}$ is the optimization variable. The inequality constraint $G\,x\preccurlyeq h$ is elementwise.

In [ ]:
import cvxpy as cp
import numpy as np

# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
P = np.random.randn(n, n)
P = P.T@P
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G@np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(1 / 2 * cp.quad_form(x, P) + q.T@x),
                 [G@x <= h,
                  A@x == b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
#print("A dual solution corresponding to the inequality constraints is")
#print(prob.constraints[0].dual_value)

### Your Turn: Water-Filling Problem in Information Theory

- Aim: allocating power to a set of $n$ channels to maximise the total channel capacity
- $x_i$, $i=1,2,\ldots, n$ represents the power allocated to the $i$-th channel
- $\log(\alpha_i + x_i)$ gives the capacity of the $i$-th channel; $\alpha_i$ is given
- Constraints: $\ds\sum_{i=1}^n x_i = 1$, $x_i\geqslant 0$
- An example
  - Given $\ds\alpha = [0.8,\,1.0,\,1.2],\;$ the maximum channel capacity: $\;0.863,\;$ power allocation: $\;[0.533,\,0.333,\,0.133]$ 

In [ ]:
import cvxpy as cp
import numpy as np
np.set_printoptions(precision=3)

#alpha = np.array([1.1, 0.7, 0.25, 0.42, 0.53, 0.31, 0.53, 0.6, 1.05, 0.27])
alpha = np.array([0.8, 1.0, 1.2])

# TODO

prob.solve()
print("Optimal value:", prob.value, "\nx:", x.value)

### Your Turn: The Maximum Entropy Problem
Formulate and solve the maximum entropy problem:
\begin{align*}
  \text{maximize}\quad & -\sum_{i = 1}^n x_i\log{x_i} \\
  \text{subject to}\quad & A x = b, \quad F x\succcurlyeq g
\end{align*}
with the following $A$, $b$, $F$, and $g$. The optimal value should be $5.480901$.

In [ ]:
import cvxpy as cp
import numpy as np

# Make random input repeatable. 
np.random.seed(0) 

# Matrix size parameters.
n = 20
m = 10
p = 5

# Generate random problem data.
tmp = np.random.rand(n)
A = np.random.randn(m, n)
b = A.dot(tmp)
F = np.random.randn(p, n)
g = F.dot(tmp) + np.random.rand(p)

In [ ]:
# TODO: Your code here

print("\nThe optimal value is:", prob.value)
print('\nThe optimal solution is:')
print(x.value)

### Your Turn: Disciplined Convex Programming (DCP) Test

[Disciplined Convex Programming: Rules, Functions, and Quizzes](https://fenchel.stanford.edu/home)

Fix optimization problems that break the DCP rules by identifying the DCP error and then rewriting the problem.

1. $\ds\min\Big\{\sqrt{x^2 + 1}\;\Big|\;x\in\mathbb{R}\Big\}$

In [ ]:
import numpy as np
import cvxpy as cp

x = cp.Variable()
prob = cp.Problem(cp.Minimize(cp.sqrt(cp.square(x) + 1)))
print(prob.is_dcp())
#prob.solve()

2. $\ds\min\Big\{x + 2\;\Big|\; 5 = \frac{2}{x},~~  x > 0\Big\}$

In [ ]:
x = cp.Variable()
prob = cp.Problem(cp.Minimize(x + 2), [5 == 2 * cp.inv_pos(x)])
print(prob.is_dcp())
#prob.solve()

3. $\ds\min\Big\{ x + 2\;\Big|\;5\leqslant\frac{2}{x^2},\,x \in \mathbb{R}\Big\}$

In [ ]:
x = cp.Variable()
prob = cp.Problem(cp.Minimize(x + 2), [5 <= 2 / cp.square(x)])
print(prob.is_dcp())
#prob.solve()

4. $\ds\min\Big\{ \frac{1}{x}\;\Big|\;0 \leqslant \frac{x^2}{y}, ~~ y \geqslant 1, ~~ x > 0 \Big\}$

In [ ]:
x = cp.Variable(pos=True)
y = cp.Variable()
prob = cp.Problem(cp.Minimize(cp.inv_pos(x)), 
                  [0 <= cp.quad_over_lin(x, y), y >= 1])
print(prob.is_dcp())
#prob.solve()

5. $\ds\min\Big\{ x + 2\;\Big|\; \exp(2x) + \exp(3x) \leqslant \exp(5x),\,\, x\in\mathbb{R}\Big\}$

In [ ]:
x = cp.Variable()
prob = cp.Problem(cp.Minimize(x + 2), [cp.exp(2 * x) + cp.exp(3 * x) <= cp.exp(5 * x)])
print(prob.is_dcp())
#prob.solve()

6. $\ds\max\Big\{-\big(\max\{x, 4\} - 3\big)^2 \;\Big|\; x \geqslant 1 \Big\}$

In [ ]:
x = cp.Variable()
prob = cp.Problem(cp.Maximize(-cp.square(cp.maximum(x, 4) - 3)), [x >= 1])
print(prob.is_dcp())
#prob.solve()

7. (Bonus) $\ds\min\bigg\{ \sum_{i=1}^m c_i\,\frac{x_i}{u_i - x_i}\;\bigg|\; ~  u > x;~~ x \in \mathbb{R}^m \bigg\}$, where $c$ and $u$ are nonnegative vectors.

In [ ]:
m = 10
np.random.seed(1)
c = np.random.randn(m)
c = np.abs(c)  
u = np.random.randn(m)
u = np.abs(u) # ensure that c and u are nonnegative

x = cp.Variable(m)
prob = cp.Problem(cp.Minimize(sum([c[i] * x[i] * cp.inv_pos(u[i] - x[i]) for i in range(m)])))
print(prob.is_dcp())
#prob.solve()